In [6]:
import networkx as nx
import numpy as np
import sys
import shapely

In [79]:
import geopandas as gpd

In [4]:
sys.path.append("/Users/haleighwright/Desktop/NCEAS/pyriv")
sys.path.append("/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages")

In [7]:
from pyriv import graph_prep as GraphBuilder
from pyriv import river_graph as RiverGraph

In [10]:
kusko_g = GraphBuilder.GraphBuilder('../../data/sasap/kusko_region_flowlines.gpickle')

do attracting components for now to get the ends of the graph

In [11]:
g_ends = nx.attracting_components(kusko_g.graph)


In [12]:
copy = kusko_g.graph.copy()


In [13]:
NODELIST = kusko_g.graph.nodes()

In [14]:
def deadends(self):
    """
    For a directed graph, find the nodes that have in edges but no out edges.
    """
    thedict = {}
    # get a dictionary of { node: out_degree }
    degdict = self.out_degree(NODELIST)
    # convert to a 2 column array of node and out_degree
    degarr = np.array(degdict.items())
    # get a boolean index of rows where out_degree==0
    dead_row_ind = (degarr[:,1]==0)
    
    for i in range(0, len(degdict)):
        if dead_row_ind[i] == True:
            thedict[i] = tuple(degarr[:,0][i])
            
    # use that index to get the nodes
    #dead_nodes = degarr[:,0][dead_row_ind]
    return thedict

In [15]:
 end_dict = deadends(kusko_g.graph)

In [106]:
end_dict_snapped = deadends(kusko_g_snapped.graph)

In [16]:
print(end_dict.items()[0])

(13, (-471290.9307224038, 1213112.07627931))


In [19]:
allnodesdict = {}
thenodes = NODELIST
for i in range(0, len(thenodes)):
    allnodesdict[i] = thenodes[i]

In [107]:
allnodesdict_snapped = {}
thenodes_snapped = NODELIST_snapped
for i in range(0, len(thenodes_snapped)):
    allnodesdict_snapped[i] = thenodes_snapped[i]

In [20]:
allnodesdict.items()[0]

(0, (-71704.67681649092, 1377383.849704166))

In [108]:
allnodesdict_snapped.items()[0]

(0, (-71704.67681649092, 1377383.849704166))

In [ ]:
#add_missing_edges(copy, end_dict, allnodesdict, 5.0 )

this process (the node snapping) could be broken up and done in parallel to speed it up...

### probably would be easiest to add an ID to nodes when creating the graph to not mess with lists

In [21]:
def add_missing_edges(graph, deadends, allnodes, threshold):
    missing_edges = []
    
    for key1, node1 in deadends.iteritems():
        for key2, node2 in allnodes.iteritems():
            #get the distance b/w nodes, if below threshold, add edge b/w them in graph
            if key1 != key2 and np.linalg.norm(np.subtract(node1,node2)) < threshold:
                graph.add_edge(key1,key2)
                print("added edge from "+str(key1)+" : "+str(node1)+" to "+ str(key2)+" : "+str(node2))
                
    
    

### add ID's to all the nodes

In [22]:
g_ID = kusko_g.graph.copy()

In [23]:
keys = g_ID.nodes()
d = g_ID.number_of_nodes()

In [87]:
 print type(g_ID).__name__

RiverGraph


In [24]:
attr_dict = {}
i = 0
for node in keys:
    attr_dict[node] = i
    i = i + 1

In [25]:
nx.set_node_attributes(g_ID, 'ID', attr_dict)

In [26]:
ids = nx.get_node_attributes(g_ID, 'ID')

In [27]:
for key,value in ids.iteritems():
    print key 
    print value
    break
    

(-71704.67681649092, 1377383.849704166)
0


### rewrite the missing edges function to rely on graph ID's & be more efficient

In [38]:
%%time
add_missing_edges_f(g_ID, end_dict, allnodesdict, 3.0 )

itr 13
itr 229924
itr 132061
itr 230919
itr 231567
itr 264901
itr 68846
itr 3965
itr 233816
itr 168789
itr 103757
itr 235362
added edge from 203007 : (-290938.300311207, 1301283.3149657596) to 246131 : (-290938.32268009393, 1301283.3127322474)
itr 170431
itr 269372
itr 269917
itr 172140
itr 8876
itr 75015
itr 239518
itr 207289
itr 76798
itr 56817
itr 110665
itr 171443
itr 144416
itr 177755
itr 238354
itr 244593
itr 139170
added edge from 144703 : (-421500.2361574562, 1149075.1478681166) to 165126 : (-421498.1604211468, 1149077.092522459)
itr 16560
itr 49918
itr 270573
added edge from 50869 : (-525261.9544729243, 1157991.6321857257) to 162910 : (-525264.6820633454, 1157991.8895438928)
added edge from 214903 : (-479747.76665325527, 1197392.2160671172) to 270210 : (-479749.8061004414, 1197391.509488749)
itr 83935
itr 51796
itr 248993
itr 184046
itr 184715
itr 87003
itr 22003
itr 251998
itr 252579
itr 56702
itr 221068
itr 254420
itr 255146
itr 26366
itr 27043
itr 93267
itr 192175
itr 43045

In [109]:
%%time
add_missing_edges_f(g_ID_snapped, end_dict_snapped, allnodesdict_snapped, 3.0 )

itr 65540
itr 33314
itr 33789
itr 99914
itr 231586
itr 2774
itr 36120
itr 69519
itr 70031
itr 234396
itr 5568
itr 6112
itr 170470
itr 40040
itr 237170
itr 66917
itr 238228
itr 271568
itr 206676
itr 198377
itr 43924
itr 208303
itr 176099
itr 143811
itr 111576
added edge from 144666 : (-421500.2361574562, 1149075.1478681166) to 166612 : (-421498.1604211468, 1149077.092522459)
itr 210551
itr 243974
itr 47922
itr 81306
itr 245851
itr 199453
itr 17745
added edge from 50857 : (-525261.9544729243, 1157991.6321857257) to 164345 : (-525264.6820633454, 1157991.8895438928)
added edge from 214842 : (-479747.76665325527, 1197392.2160671172) to 270457 : (-479749.8061004414, 1197391.509488749)
itr 83858
itr 84524
itr 183339
itr 20128
itr 250147
itr 250784
itr 120218
itr 219167
itr 23191
itr 89485
itr 90019
itr 25084
itr 91296
itr 255766
itr 256412
itr 191536
itr 126583
itr 94373
itr 256129
itr 30046
itr 30660
itr 31302
itr 64688
itr 130846
CPU times: user 59min 3s, sys: 3min 51s, total: 1h 2min 55s
W

In [37]:
def add_missing_edges_f(graph, deadends, allnodes, threshold):
    cnt = 0
    for key1, node1 in deadends.iteritems():
        if cnt%100==0:
            print("itr "+str(key1))
        cnt = cnt + 1
        for key2, node2 in allnodes.iteritems():
            #print node1
            xd = node1[0] - node2[0]
            yd = node1[1] - node2[1]
            dist = np.sqrt((xd*xd + yd*yd))
            
            if key1 != key2 and dist < threshold and dist > 0:
                graph.add_edge(key1, key2)
                print("added edge from "+str(key1)+" : "+str(node1)+" to "+ str(key2)+" : "+str(node2))
                break
                
            
    

In [64]:
nx.write_shp(g_ID,'../../data/sasap')

TypeError: 'int' object has no attribute '__getitem__'

In [78]:
for key, val in g_ID.edges():
    print type(key).__name__
    print key
    print value
    break

KeyboardInterrupt: 

## cpu bound --> need multiprocessing for parallel

In [56]:
from multiprocessing import Pool

In [81]:
%%time
pool = Pool(processes=5)
pool.map(add_missing_edges_p, end_dict.items())
pool.close()
pool.join()


AttributeError: 'tuple' object has no attribute 'iteritems'

In [62]:
end_dict.items()[0]

(13, (-471290.9307224038, 1213112.07627931))

In [ ]:
pool.apply_async(add_missing_edges_p,)

In [57]:

cnt = 0
allnodes = allnodesdict
graph = g_ID
threshold = 5.0
    
for key1, node1 in deadends.iteritems():
    if cnt%100==0:
        print("itr "+str(key1))
    cnt = cnt + 1
    for key2, node2 in allnodes.iteritems():
        #print node1
        xd = node1[0] - node2[0]
        yd = node1[1] - node2[1]
        dist = np.sqrt((xd*xd + yd*yd))
            
        if key1 != key2 and dist < threshold and dist > 0:
            graph.add_edge(key1, key2)
            print("added edge from "+str(key1)+" : "+str(node1)+" to "+ str(key2)+" : "+str(node2))
            break

In [ ]:
def add_missing_edges_p(deadends):
    

### get some stats on the snapped network

In [88]:
kusko_g_snapped = GraphBuilder.GraphBuilder('../../data/sasap/kusko_region_flowlines_snap.shp')

In [89]:
g_ID_snapped = kusko_g_snapped.graph.copy()
print type(g_ID_snapped).__name__    

RiverGraph


In [90]:
keys = g_ID_snapped.nodes()
d = g_ID_snapped.number_of_nodes()
attr_dict = {}
i = 0
for node in keys:
    attr_dict[node] = i
    i = i + 1
nx.set_node_attributes(g_ID_snapped, 'ID', attr_dict)
ids = nx.get_node_attributes(g_ID_snapped, 'ID')
for key,value in ids.iteritems():
    print key 
    print value
    break

(-71704.67681649092, 1377383.849704166)
0


In [110]:
print nx.number_weakly_connected_components(g_ID)
print nx.number_weakly_connected_components(g_ID_snapped)

5552
5551


In [111]:
print nx.number_weakly_connected_components(kusko_g.graph)
print  nx.number_weakly_connected_components(kusko_g_snapped.graph)

5548
5548


In [112]:
print nx.is_weakly_connected(g_ID)
print nx.is_weakly_connected(g_ID_snapped)

False
False


In [113]:
print nx.is_weakly_connected(kusko_g.graph)
print nx.is_weakly_connected(kusko_g_snapped.graph)

False
False


In [114]:
print nx.is_semiconnected(g_ID)
print nx.is_semiconnected(g_ID_snapped)

False
False


In [115]:
print nx.is_semiconnected(kusko_g.graph)
print nx.is_semiconnected(kusko_g_snapped.graph)

False
False


In [116]:
print g_ID.number_of_edges()
print g_ID_snapped.number_of_edges()

284701
284690


In [99]:
print kusko_g.graph.number_of_edges()
print kusko_g_snapped.graph.number_of_edges()

284697
284687


In [117]:
tmp_g = g_ID.to_undirected()
tmp_g_s = g_ID_snapped.to_undirected()

In [118]:
tmp_k = kusko_g.graph.to_undirected()
tmp_k_s = kusko_g.graph.to_undirected()


In [119]:
print nx.is_connected(tmp_g)
print nx.is_connected(tmp_k)
print nx.is_connected(tmp_g_s)
print nx.is_connected(tmp_k_s)

False
False
False
False


In [104]:
NODELIST_snapped = kusko_g_snapped.graph.nodes()

In [105]:
print NODELIST_snapped[(-290938.300311207, 1301283.3149657596)]

TypeError: list indices must be integers, not tuple

## some visualizations with geopandas

In [80]:
orig = gpd.read_file('../../data/sasap/kusko_region_flowlines.shp')